# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to lidalong0408@126.com and will expire on October 30, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-8989 - Server binary: /Users/lidalong/anaconda2/envs/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1458198489.log
[INFO] GraphLab Server Version: 1.8


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.567620     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.616081     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [6]:
print model_all.get('coefficients').print_rows(17)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [8]:
import numpy as np

min_rss = None
best_l1_penalty = None
for l1_penalty in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predict_value = model.predict(validation)
    rss = ((validation['price'] - predict_value)**2).sum()
    if min_rss is None or rss < min_rss:
        min_rss = rss
        best_l1_penalty = l1_penalty

l1_penalty=1.00e+01 produced the lowest RSS(6.258e+14)


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [9]:
print "best value for the l1_penalty: %.2e" % best_l1_penalty
best_model = graphlab.linear_regression.create(testing, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=best_l1_penalty)
test_predict = best_model.predict(testing)
test_rss = ((test_predict - testing['price'])**2).sum()
print "RSS on TEST data of the model with the best l1_penalty: %.3e" % test_rss

best value for the l1_penalty: 1.00e+01
RSS on TEST data of the model with the best l1_penalty: 1.552e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [10]:
print best_model.get('coefficients').print_rows(17)

+------------------+-------+----------------+--------+
|       name       | index |     value      | stderr |
+------------------+-------+----------------+--------+
|   (intercept)    |  None | 15938.3491302  |  None  |
|     bedrooms     |  None | 7927.96151068  |  None  |
| bedrooms_square  |  None |  2811.6551531  |  None  |
|    bathrooms     |  None |  25090.812508  |  None  |
|   sqft_living    |  None | 39.6894164796  |  None  |
| sqft_living_sqrt |  None | 1115.10438309  |  None  |
|     sqft_lot     |  None | -0.15538020793 |  None  |
|  sqft_lot_sqrt   |  None | 126.735040031  |  None  |
|      floors      |  None | 19806.9697903  |  None  |
|  floors_square   |  None | 12646.6371612  |  None  |
|    waterfront    |  None | 413704.286579  |  None  |
|       view       |  None | 86807.2319587  |  None  |
|    condition     |  None | 5914.49546888  |  None  |
|      grade       |  None | 6017.72238936  |  None  |
|    sqft_above    |  None | 44.5451590346  |  None  |
|  sqft_ba

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [11]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [12]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [13]:
weights_nnz = []
for l1_penalty in l1_penalty_values:
    print l1_penalty
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    weights_nnz.append(model['coefficients']['value'].nnz())
print weights_nnz

100000000.0
127427498.57
162377673.919
206913808.111
263665089.873
335981828.628
428133239.872
545559478.117
695192796.178
885866790.41
1128837891.68
1438449888.29
1832980710.83
2335721469.09
2976351441.63
3792690190.73
4832930238.57
6158482110.66
7847599703.51
10000000000.0
[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [14]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [15]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [22]:
min_rss = None
l1_penalty_max_nonzeros = None
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predict = model.predict(validation)
    print "l1_penalty: %f nnz: %d" % (l1_penalty, model['coefficients']['value'].nnz())
    
    if model['coefficients']['value'].nnz() == max_nonzeros:
        test_rss = ((predict - validation['price'])**2).sum()
        if l1_penalty_max_nonzeros is None or test_rss < min_rss:
            min_rss = test_rss
            l1_penalty_max_nonzeros = l1_penalty
        
print min_rss

print l1_penalty_max_nonzeros

l1_penalty: 2976351441.630000 nnz: 10
l1_penalty: 3019316638.951053 nnz: 10
l1_penalty: 3062281836.272105 nnz: 10
l1_penalty: 3105247033.593158 nnz: 10
l1_penalty: 3148212230.914211 nnz: 10
l1_penalty: 3191177428.235263 nnz: 10
l1_penalty: 3234142625.556316 nnz: 10
l1_penalty: 3277107822.877368 nnz: 10
l1_penalty: 3320073020.198421 nnz: 8
l1_penalty: 3363038217.519474 nnz: 8
l1_penalty: 3406003414.840527 nnz: 8
l1_penalty: 3448968612.161579 nnz: 7
l1_penalty: 3491933809.482632 nnz: 7
l1_penalty: 3534899006.803684 nnz: 7
l1_penalty: 3577864204.124737 nnz: 7
l1_penalty: 3620829401.445789 nnz: 6
l1_penalty: 3663794598.766842 nnz: 6
l1_penalty: 3706759796.087895 nnz: 6
l1_penalty: 3749724993.408947 nnz: 6
l1_penalty: 3792690190.730000 nnz: 6
1.04693748875e+15
3448968612.16


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [23]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=validation, verbose=False,
                                              l2_penalty=0., l1_penalty=l1_penalty_max_nonzeros)
model.get('coefficients').print_rows(20)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |